# TaskTree Tutorial

In this tutorial we will walk through the capabilities of task trees in pycram.

First we have to import the necessary functionality from pycram.

In [1]:
from pycram.bullet_world import BulletWorld
from pycram.robot_descriptions.robot_description_handler import InitializedRobotDescription as robot_description
import pycram.task
from pycram.resolver.plans import Arms
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.designators.object_designator import *
import anytree
import pycram.plan_failures

pybullet build time: Sep 20 2021 20:33:29
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


Next we will create a bullet world with a PR2 in a kitchen containing milk and cereal.

In [2]:
world = BulletWorld()
robot = Object(robot_description.i.name, "robot", robot_description.i.name + ".urdf")
apartment = Object("apartment", "environment", "/home/abassi/cram_ws/src/iai_maps/iai_apartment/urdf/apartment.urdf", position=[-1.5, -2.5, 0])
table_top = apartment.get_link_position("cooktop")
# milk = Object("milk", "milk", "milk.stl", position=[table_top[0]-0.15, table_top[1], table_top[2]])
# milk.set_position(position=milk.get_position(), base=True)
# cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=table_top)
# cereal.set_position(position=[table_top[0]-0.1, table_top[1] + 0.5, table_top[2]], base=True)
# milk_desig = ObjectDesignator(ObjectDesignatorDescription(name="milk", type="milk"))
# cereal_desig = ObjectDesignator(ObjectDesignatorDescription(name="cereal", type="cereal"))

In [3]:
import numpy as np

def get_n_random_positions(pose_list, n=4, dist=0.5):
    positions = [pos[0] for pos in pose_list]
    all_indices = list(range(len(positions)))
    # pos_idx = np.random.choice(all_indices)
    pos_idx = all_indices[0]
    all_indices.remove(pos_idx)
    n_positions = np.zeros((n,3))
    for i in range(n):
        n_positions[i,:] = positions[pos_idx]
    found_count = 1
    found_indices = [pos_idx]
    for i in range(len(positions)-1):
        # pos_idx = np.random.choice(all_indices)
        pos_idx = all_indices[i]
        diff = np.absolute(np.linalg.norm(n_positions - positions[pos_idx], axis=1))
        # print(diff)
        min_diff = np.min(diff)
        # print(min_diff)
        if min_diff >= dist:
            # print("found")
            n_positions[found_count,:] = positions[pos_idx]
            found_indices.append(pos_idx)
            found_count += 1
        all_indices.remove(pos_idx)
        if found_count == n:
            break
    found_poses = [pose_list[i] for i in found_indices]
    # found_positions = [positions[i] for i in found_indices]
    # for i in range(len(found_positions)):
    #     print(found_poses[i][0])
    #     print(found_positions[i])
    #     assert np.allclose(found_positions[i],found_poses[i][0])
    # for i in range(len(found_poses)):
    #     for j in range(i+1,len(found_poses)):
    #         pos1 = np.array(found_poses[i][0])
    #         pos2 = np.array(found_poses[j][0])
    #         diff = np.absolute(np.linalg.norm(pos1 - pos2))
    #         print(diff)
    #         assert diff >= dist
    return found_poses




In [5]:
from pycram.costmaps import SemanticCostmap
from pycram.pose_generator_and_validator import pose_generator
scm = SemanticCostmap(apartment,"island_countertop")
poses_list = list(pose_generator(scm))
poses_list.sort(reverse=True, key=lambda x: np.linalg.norm(x[0]))
object_poses = get_n_random_positions(poses_list)
object_names = ["bowl", "milk", "breakfast_cereal", "spoon"]
objects = {}
object_desig = {}
for obj_name, obj_pose in zip(object_names, object_poses):
    print(obj_name)
    print(obj_pose)
    objects[obj_name] = Object(obj_name, obj_name, obj_name+".stl", position=[obj_pose[0][0], obj_pose[0][1], table_top[2]])
    objects[obj_name].move_base_to_origin_pos()
    objects[obj_name].original_pose = objects[obj_name].get_position_and_orientation()
    object_desig[obj_name] = ObjectDesignator(ObjectDesignatorDescription(name=obj_name, type=obj_name))
print(object_poses)

bowl
([1.389640212059021, 1.7239844799041748, 0.921999990940094], [-0.0, 0.0, 0.7385697797052483, -0.6741770394385594])
milk
([1.3888438940048218, 1.2239850759506226, 0.921999990940094], [-0.0, 0.0, 0.7525015983779697, -0.6585904223708395])
breakfast_cereal
([1.4052444696426392, -1.0360441207885742, 0.921999990940094], [0.0, 0.0, 0.7518593655690526, 0.6593235127053347])
spoon
([1.3880475759506226, 0.7239856719970703, 0.921999990940094], [-0.0, 0.0, 0.7886953551919237, -0.6147842196239957])
[([1.389640212059021, 1.7239844799041748, 0.921999990940094], [-0.0, 0.0, 0.7385697797052483, -0.6741770394385594]), ([1.3888438940048218, 1.2239850759506226, 0.921999990940094], [-0.0, 0.0, 0.7525015983779697, -0.6585904223708395]), ([1.4052444696426392, -1.0360441207885742, 0.921999990940094], [0.0, 0.0, 0.7518593655690526, 0.6593235127053347]), ([1.3880475759506226, 0.7239856719970703, 0.921999990940094], [-0.0, 0.0, 0.7886953551919237, -0.6147842196239957])]


If You want to visualize all apartment frames

In [6]:
import pybullet as p
for link_name in apartment.links.keys():
    world.add_vis_axis(apartment.get_link_position_and_orientation(link_name))
    p.addUserDebugText(link_name, apartment.get_link_position(link_name))

In [7]:
world.remove_vis_axis()
p.removeAllUserDebugItems()

Finally, we create a plan where the robot parks his arms, walks to the kitchen counter and picks the thingy. Then we execute the plan.

In [6]:

@pycram.task.with_tree
def plan(obj, obj_desig, torso=0.2, place="countertop"):
    world.reset_bullet_world()
    with simulated_robot:
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        ActionDesignator(MoveTorsoAction(torso)).perform()

        location = LocationDesignator(CostmapLocation(target=obj, reachable_for=robot))
        pose = location.reference()

        ActionDesignator(
            NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()
        good_torsos.append(torso)
        picked_up_arm = pose["arms"][0]
        ActionDesignator(PickUpAction(object_designator=obj_desig, arm=pose["arms"][0], grasp="front")).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()
        scm = SemanticCostmapLocation(place, apartment, obj_desig.prop_value("object"))
        place_island = LocationDesignator(scm)
        pose_island = place_island.reference()

        place_location = LocationDesignator(CostmapLocation(target=list(pose_island.values()), reachable_for=robot, reachable_arm=picked_up_arm))
        pose = place_location.reference()

        ActionDesignator(NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()

        ActionDesignator(PlaceAction(object_designator=obj_desig, target_location=list(pose_island.values()), arm=picked_up_arm)).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

good_torsos = []
for obj_name in object_names:
    done = False
    torso = 0.25 if len(good_torsos) == 0 else good_torsos[-1]
    while not done:
        try:
            plan(objects[obj_name], object_desig[obj_name], torso=torso, place="island_countertop")
            done = True
            objects[obj_name].original_pose = objects[obj_name].get_position_and_orientation()
        except:
            print("no solution")
            torso += 0.05
            if torso > 0.3:
                break
print(good_torsos)

Parking arms Arms.BOTH.
Moving to [1.5893018245697021, 2.246276617050171, 0.0]. Orientation: [-0.0, 0.0, 0.8243115669267845, -0.5661364152134265].
(1.5893018245697021, 2.246276617050171, 0.0)
Parking arms Arms.BOTH.
Picking up ObjectDesignator({'resolver': 'grounding', 'type': 'bowl', 'name': 'bowl', 'object': None, 'pose': None}) with right.
Parking arms Arms.BOTH.
Moving to [2.1486527919769287, 1.163985252380371, 0.0]. Orientation: [-0.0, 0.0, 0.9992242377126355, -0.039381756786647744].
(2.1486527919769287, 1.163985252380371, 0.0)
Placing ObjectDesignator({'resolver': 'grounding', 'type': 'bowl', 'name': 'bowl', 'object': <pycram.bullet_world.Object object at 0x7f1e2755afd0>, 'pose': ((1.3893018253147602, 1.726276684552431, 0.979470837533474), (0.0, 0.0, 0.0, 1.0))}) with right at [[1.3886528015136719, 1.10398530960083, 0.9614115809914588], [-0.0, 0.0, 0.7579203761145179, -0.6523470728610865]].
Parking arms Arms.BOTH.
Parking arms Arms.BOTH.
no solution
Parking arms Arms.BOTH.
no sol

Now we get the task tree from its module and render it. Rendering can be done with any render method described in the anytree package. We will use ascii rendering here for ease of displaying.

In [4]:
tt = pycram.task.task_tree
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

no_operation()
+-- plan()
    |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5ea8453a30>.park_arms(arms = Arms.BOTH)
    |-- <pycram.designators.action_designator.NavigateAction object at 0x7f5ea8453f70>.navigate(target = [0.7199999690055847, 1.0399999618530273, 0.0], orientation = [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606])
    |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5ea8453d30>.park_arms(arms = Arms.BOTH)
    |-- <pycram.designators.action_designator.PickUpAction object at 0x7f5e302ca280>.pick_up(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), grasp = front)
    |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5e302ca730>.park_arms(arms = Arms.BOTH)
    |-- <pycram.designators.action_designator.NavigateAction object at 0x7f5e302ca

As we see every task in the plan got recorded correctly. It is noticeable that the tree begins with a NoOperation node. This is done because several, not connected, plans that get executed after each other should still appear in the task tree. Hence, a NoOperation node is the root of any tree. If we re-execute the plan we would see them appear in the same tree even though they are not connected.

In [5]:
world.reset_bullet_world()
plan()
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

Parking arms Arms.BOTH.
Moving to [0.7199999690055847, 1.0399999618530273, 0.0]. Orientation: [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606].
(0.7199999690055847, 1.0399999618530273, 0.0)
Parking arms Arms.BOTH.
Picking up ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left.
Not attached to anything!
Parking arms Arms.BOTH.
Moving to [-1.9075000286102295, 0.7792000770568848, 0.0]. Orientation: [0.0, 0.0, 0.16439898301071468, 0.9863939245479175].
(-1.9075000286102295, 0.7792000770568848, 0.0)
Placing ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left at [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]].
Parking arms Arms.BOT

Projecting a plan in a new environment with its own task tree that only exists while the projected plan is running can be done with the ``with`` keyword. When this is done, both the bullet world and task tree are saved and new, freshly reset objects are available. At the end of a with block the old state is restored. The root for such things is then called ``simulation()``.

In [6]:
with pycram.task.SimulatedTaskTree() as stt:
    print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

simulation()
no_operation()
|-- plan()
|   |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5ea8453a30>.park_arms(arms = Arms.BOTH)
|   |-- <pycram.designators.action_designator.NavigateAction object at 0x7f5ea8453f70>.navigate(target = [0.7199999690055847, 1.0399999618530273, 0.0], orientation = [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606])
|   |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5ea8453d30>.park_arms(arms = Arms.BOTH)
|   |-- <pycram.designators.action_designator.PickUpAction object at 0x7f5e302ca280>.pick_up(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), grasp = front)
|   |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5e302ca730>.park_arms(arms = Arms.BOTH)
|   |-- <pycram.designators.action_designator.NavigateAction object a

Task tree can be manipulated with ordinary anytree manipulation. If we for example want to discard the second plan, we would write

In [7]:
tt.root.children = (tt.root.children[0],)
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

no_operation()
+-- plan()
    |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5ea8453a30>.park_arms(arms = Arms.BOTH)
    |-- <pycram.designators.action_designator.NavigateAction object at 0x7f5ea8453f70>.navigate(target = [0.7199999690055847, 1.0399999618530273, 0.0], orientation = [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606])
    |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5ea8453d30>.park_arms(arms = Arms.BOTH)
    |-- <pycram.designators.action_designator.PickUpAction object at 0x7f5e302ca280>.pick_up(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), grasp = front)
    |-- <pycram.designators.action_designator.ParkArmsAction object at 0x7f5e302ca730>.park_arms(arms = Arms.BOTH)
    |-- <pycram.designators.action_designator.NavigateAction object at 0x7f5e302ca

We can now re-execute this (modified) plan by executing the leaf in pre-ordering iteration using the anytree functionality. This will not append the re-execution to the task tree.

In [8]:
world.reset_bullet_world()
with simulated_robot:
    [node.code.execute() for node in tt.root.leaves]
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

Parking arms Arms.BOTH.
Moving to [0.7199999690055847, 1.0399999618530273, 0.0]. Orientation: [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606].
(0.7199999690055847, 1.0399999618530273, 0.0)
Parking arms Arms.BOTH.
Picking up ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left.
Not attached to anything!
Parking arms Arms.BOTH.
Moving to [-1.9075000286102295, 0.7792000770568848, 0.0]. Orientation: [0.0, 0.0, 0.16439898301071468, 0.9863939245479175].
(-1.9075000286102295, 0.7792000770568848, 0.0)
Placing ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f5ea8413eb0>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left at [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]].
Parking arms Arms.BOT

Nodes in the task tree contain additional information about the status and time of a task.

In [9]:
print(pycram.task.task_tree.children[0])

Code: plan() 
 start_time: 2023-04-13 10:49:43.184007 
 Status: TaskStatus.SUCCEEDED 
 end_time: 2023-04-13 10:49:47.980515 
 


The task tree can also be reset to an empty one by invoking

In [10]:
pycram.task.reset_tree()
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

no_operation()


If a plan fails using the PlanFailure exception, the plan will not stop. Instead, the error will be logged and saved in the task tree as a failed subtask. First let's create a simple failing plan and execute it.

In [11]:
@pycram.task.with_tree
def failing_plan():
    raise pycram.plan_failures.PlanFailure("Oopsie!")

failing_plan()

We can now investigate the nodes of the tree, and we will see that the tree indeed contains a failed task.

In [12]:
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))
print(pycram.task.task_tree.children[0])

no_operation()
+-- failing_plan()
Code: failing_plan() 
 start_time: 2023-04-13 10:50:27.493840 
 Status: TaskStatus.FAILED 
 end_time: 2023-04-13 10:50:27.494815 
 


In [1]:
world.exit()

NameError: name 'world' is not defined